In [1]:
import config as c 
from src.data import CombiDataset,make_loader,downsampling
from src.model import Model 
from src.freia_funcs import * 
from src.utils import * 
from tqdm import tqdm 
import tifffile  
import os 
import pandas as pd 
import torch.nn.functional as F 


my_experiment


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train Teacher 

In [ ]:
from train_teacher import train as train_teacher 
all_classes = [d for d in os.listdir(c.dataset_dir) if os.path.isdir(os.path.join(c.dataset_dir, d))]
all_classes.remove('.ipynb_checkpoints')
all_classes.remove('split_csv')

dataset_dir = c.dataset_dir
mode = 'feature'
train = 'train'

#for class_name in all_classes:
for class_name in ['cable']:
    print(f'\n Class : {class_name}')
    save_dir = os.path.join('./saved_models',dataset_dir.split('/')[-1],class_name)
    
    trainset = CombiDataset(dataset_dir,class_name,mode,train,device=c.device)
    trainloader = make_loader(trainset,shuffle=True)
    
    teacher = Model().to(c.device)
    train_teacher(teacher,trainloader,save_dir)

In [ ]:
from train_teacher import train as train_teacher 

class_name = 'cable'
mode = c.data_mode
train='train'
save_dir = os.path.join('./saved_models',c.dataset_dir.split('/')[-1],class_name)
trainset = CombiDataset(c.dataset_dir,class_name,mode,train,device=c.device)
trainloader = make_loader(trainset,shuffle=True)

teacher = torch.load(f'./saved_models/MVtecAD/{class_name}/teacher_best.pt').to(c.device)
train_teacher(teacher,trainloader,save_dir)

# Train Student 

In [ ]:
from train_student import train as train_student
all_classes = [d for d in os.listdir(c.dataset_dir) if os.path.isdir(os.path.join(c.dataset_dir, d))]
all_classes.remove('.ipynb_checkpoints')
all_classes.remove('split_csv')
#for class_name in all_classes:
for class_name in ['bottle']:
    print(f'\t Class : {class_name}')
    save_dir = os.path.join('./saved_models',dataset_dir.split('/')[-1],class_name) 
    teacher = torch.load(f'./saved_models/MVtecAD/{class_name}/teacher_best.pt').to(c.device)
    student = Model(nf=not c.asymmetric_student, channels_hidden=c.channels_hidden_student, n_blocks=c.n_st_blocks).to(c.device)
    train_student(teacher,student,trainloader,save_dir)

# Eval 

In [1]:
import config as c 
from src.data import CombiDataset,make_loader,downsampling
from src.model import Model 
from src.freia_funcs import * 
from src.utils import * 
from tqdm import tqdm 
import tifffile  
import os 
import pandas as pd 
import torch.nn.functional as F 

from eval import eval 
score = {} 
all_classes = [d for d in os.listdir(c.dataset_dir) if os.path.isdir(os.path.join(c.dataset_dir, d))]
all_classes.remove('.ipynb_checkpoints')
all_classes.remove('split_csv')
dataset_dir = c.dataset_dir
mode = c.data_mode
for class_name in all_classes:
#for class_name in ['grid']:
    print(f'\n Class : {class_name}')
    save_dir = os.path.join('./saved_models',dataset_dir.split('/')[-1],class_name)


    testset = CombiDataset(dataset_dir,class_name,mode,'test',device=c.device)
    testloader = make_loader(testset)

    teacher = torch.load(f'./saved_models/MVtecAD/{class_name}/teacher_best.pt')
    student = torch.load(f'./saved_models/MVtecAD/{class_name}/student_best.pt')

    mean_img_auc,max_img_auc = eval(teacher,student,testloader)
    score[class_name] = {'max' : max_img_auc,'mean' : mean_img_auc}
    
    

my_experiment


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



 Class : leather
Loaded pretrained weights for efficientnet-b5

 Class : zipper
Loaded pretrained weights for efficientnet-b5

 Class : metal_nut
Loaded pretrained weights for efficientnet-b5

 Class : wood
Loaded pretrained weights for efficientnet-b5

 Class : pill
Loaded pretrained weights for efficientnet-b5

 Class : transistor
Loaded pretrained weights for efficientnet-b5

 Class : grid
Loaded pretrained weights for efficientnet-b5

 Class : tile
Loaded pretrained weights for efficientnet-b5

 Class : capsule
Loaded pretrained weights for efficientnet-b5

 Class : hazelnut
Loaded pretrained weights for efficientnet-b5

 Class : toothbrush
Loaded pretrained weights for efficientnet-b5

 Class : screw
Loaded pretrained weights for efficientnet-b5

 Class : carpet
Loaded pretrained weights for efficientnet-b5

 Class : bottle
Loaded pretrained weights for efficientnet-b5

 Class : cable
Loaded pretrained weights for efficientnet-b5


In [9]:
score

{'leather': {'max': 1.0, 'mean': 1.0},
 'zipper': {'max': 0.9774159663865547, 'mean': 0.9908088235294118},
 'metal_nut': {'max': 0.998533724340176, 'mean': 0.9902248289345064},
 'wood': {'max': 0.9271929824561403, 'mean': 0.7956140350877193},
 'pill': {'max': 0.70949263502455, 'mean': 0.8876159301691215},
 'transistor': {'max': 0.44166666666666665, 'mean': 0.6449999999999999},
 'grid': {'max': 0.6516290726817043, 'mean': 0.4862155388471178},
 'tile': {'max': 0.7763347763347763, 'mean': 0.9318181818181819},
 'capsule': {'max': 0.5053849222177902, 'mean': 0.4148384523334663},
 'hazelnut': {'max': 0.73, 'mean': 0.91},
 'toothbrush': {'max': 0.675, 'mean': 0.8833333333333333},
 'screw': {'max': 0.477146956343513, 'mean': 0.7511785201885632},
 'carpet': {'max': 0.9414125200642054, 'mean': 0.7608346709470305},
 'bottle': {'max': 0.5547619047619048, 'mean': 0.6428571428571428},
 'cable': {'max': 0.9190404797601199, 'mean': 0.9891304347826086}}

In [2]:
score

{'leather': {'max': 1.0, 'mean': 1.0},
 'zipper': {'max': 0.9774159663865547, 'mean': 0.9908088235294118},
 'metal_nut': {'max': 0.9956011730205279, 'mean': 0.989247311827957},
 'wood': {'max': 0.9921052631578947, 'mean': 0.987719298245614},
 'pill': {'max': 0.9639934533551554, 'mean': 0.9923622476813967},
 'transistor': {'max': 0.9870833333333333, 'mean': 0.9895833333333334},
 'grid': {'max': 0.9991645781119465, 'mean': 0.9899749373433584},
 'tile': {'max': 0.996031746031746, 'mean': 0.9992784992784993},
 'capsule': {'max': 0.9712804148384523, 'mean': 0.9924212205823694},
 'hazelnut': {'max': 0.9971428571428572, 'mean': 0.9982142857142857},
 'toothbrush': {'max': 0.8638888888888889, 'mean': 0.9611111111111111},
 'screw': {'max': 0.9442508710801394, 'mean': 0.9928263988522238},
 'carpet': {'max': 0.9719101123595505, 'mean': 0.9723113964686999},
 'bottle': {'max': 0.9944444444444445, 'mean': 0.9976190476190476},
 'cable': {'max': 0.9390929535232384, 'mean': 0.9917541229385307}}

In [4]:
mean_value = [] 
max_value = [] 
for i in score.keys():
    mean_value.append(score[i]['mean'])
    max_value.append(score[i]['max'])
print(np.mean(max_value)*100)
print(np.mean(mean_value)*100)

97.28937370449819
98.96821356350557


In [18]:
df = pd.DataFrame([mean_value,max_value]).T
df.columns = ['mean','max']
df.index = score.keys()
df

,mean,max
leather,1.000000,1.000000
zipper,0.990809,0.977416
metal_nut,0.989247,0.995601
wood,0.987719,0.992105
pill,0.992362,0.963993
transistor,0.989583,0.987083
grid,0.989975,0.999165
tile,0.999278,0.996032
capsule,0.992421,0.971280
hazelnut,0.998214,0.997143
